# TimeTrigger Example (CronTrigger)

A minimal example showing how to schedule work on a cron-like cadence using `CronTrigger`.

Why it's useful:
- Scheduling via crontab strings (e.g., "every minute", "2 PM on Mondays").
- Runs inside BitSwan's event loop; no external scheduler needed.
- Great for periodic tasks like reminders, heartbeats, or maintenance jobs.


----------------
### Imports

We import BitSwan helpers for notebook pipelines and the HTTP bits. We will also import `CronTrigger` to schedule an action every minute.


In [ ]:
from bspump.jupyter import *
from bspump.abc.source import TriggerSource
from bspump.trigger.crontrig import CronTrigger
from bspump.common.print import PPrintSink
from datetime import datetime, timezone

-----------------------
### HeartbeatSource
 
The `HeartbeatSource` is a custom trigger source that generates time-based events. It's designed to work with the `CronTrigger` to periodically inject timestamp data into the pipeline, allowing us to demonstrate scheduled processing and time-triggered workflows.

In [ ]:
class HeartbeatSource(TriggerSource):
    async def cycle(self, *args, **kwargs):
        await self.Pipeline.ready()
        await self.Pipeline.process({
            "time_triggered": datetime.now(timezone.utc).isoformat()
        })
          globals().setdefault("HEARTBEATS", []).append({"time_triggered": datetime.now().isoformat()})

-------------------
### Auto-pipeline and CronTrigger explained

We wire `HeartbeatSource` to `PPrintSink`, and attach a `CronTrigger` so the source is fired on a schedule.

Cron expressions have 5 fields: `minute hour day-of-month month day-of-week`
- `* * * * *`  every minute
- `*/10 * * * *`  every 10 minutes
- `0 14 * * 1`  every Monday at 14:00 (2 PM)
- `30 9-17 * * 1-5`  at minute 30 past each hour 9..17 on weekdays

Notes
- Warning: if you rely on local/naive time, verify the runner’s clock/timezone so schedules match your intent.
- CronTrigger fires when the cron expression is due. `.on(trigger)` attaches it to `HeartbeatSource`, which runs `cycle()` to emit an event.
- The app ticks ~once per second; CronTrigger evaluates on each tick but fires only when due.


In [ ]:
from datetime import datetime, timezone

auto_pipeline(
    source=lambda app, pipeline: HeartbeatSource(app, pipeline).on(
        CronTrigger(app, "* * * * *", init_time=datetime.now()) # Every minute
    ),
    sink=lambda app, pipeline: PPrintSink(app, pipeline),
    name="TimeTriggerPipeline"
)